In [6]:
# Fake data
import random
import csv

# Number of rows of data you want to generate
num_rows = 1000

# Range of values for each element in the row
min_value = 10
max_value = 60

# File to save the data
filename = 'fake_data.csv'

with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    for _ in range(num_rows):
        row = [random.randint(min_value, max_value) for _ in range(3)]
        writer.writerow(row)

print(f"Data generated and saved to {filename}")

Data generated and saved to fake_data.csv


In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error 



data = pd.read_csv('fake_data.csv', header=None, names=['rabbits', 'foxes', 'grass'])
print(data.head())
print(data.info())

   rabbits  foxes  grass
0       17     51     33
1       13     33     46
2       58     26     46
3       45     14     26
4       42     37     14
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   rabbits  1000 non-null   int64
 1   foxes    1000 non-null   int64
 2   grass    1000 non-null   int64
dtypes: int64(3)
memory usage: 23.6 KB
None


In [30]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

In [31]:
def create_lag_features(data, n_lags=1):
    X, y = [], []
    for i in range(len(data) - n_lags):
        X.append(data[i:i + n_lags].flatten())
        y.append(data[i + n_lags])
    return np.array(X), np.array(y)

X, y = create_lag_features(data_scaled, n_lags=1)

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [33]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)


LinearRegression()

In [38]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'MSE: {mse}')

MSE: 0.09846552735748604


In [37]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

# Assuming X_train, y_train, X_test, y_test are already defined

# Reshape input to be 3D [samples, time steps, features]
X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Define the LSTM model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(1, X_train.shape[1])))
model.add(Dense(3))
model.compile(optimizer='adam', loss='mse')

# Fit the model
model.fit(X_train_lstm, y_train, epochs=200, batch_size=32, validation_data=(X_test_lstm, y_test), verbose=0)

# Predict using the model
y_pred = model.predict(X_test_lstm)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'MSE: {mse}')

# Example of predicting the next sequence
current_sequence = data_scaled[-1].reshape(1, -1)
current_sequence_lstm = current_sequence.reshape((current_sequence.shape[0], 1, current_sequence.shape[1]))
next_sequence_scaled = model.predict(current_sequence_lstm)

# Inverse transform to original scale
next_sequence = scaler.inverse_transform(next_sequence_scaled)
print(next_sequence)


7/7 [==============================] - 0s 2ms/step
MSE: 0.09515743912752828
1/1 [==============================] - 0s 19ms/step
[[31.756622 32.200413 35.639442]]
